In [4]:
import numpy as np
import pandas as pd

from urllib.request import urlopen
import json
import plotly.express as px
from IPython.display import IFrame

# DATA

Data Dictionary: 
* https://www2.census.gov/programs-surveys/cbp/technical-documentation/records-layouts/2017_record_layouts/zip_code_industry_detail_record_layout_2017.txt

NAICS Code Meanings:
* https://www.naics.com/six-digit-naics/

In [35]:
file_path = '../../src/data/temp/zbp_totals_with_features.csv'
data = pd.read_csv(file_path)
data.head()

,zip,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,year,naics_11_pct,...,naics_99_pct,n1_4_pct,n5_9_pct,n10_19_pct,n20_49_pct,n50_99_pct,n100_249_pct,n250_499_pct,n500_999_pct,n1000_pct
0,91901,H,4141,H,36304,H,174786,391,2012,0.000000,...,0.0,0.649616,0.166240,0.107417,0.056266,0.010230,0.005115,0.002558,0.0,0.002558
1,91902,G,2265,G,19111,G,81569,349,2012,0.002865,...,0.0,0.641834,0.183381,0.094556,0.060172,0.017192,0.002865,0.000000,0.0,0.000000
2,91903,S,0,G,123,G,491,13,2012,0.000000,...,0.0,0.923077,0.076923,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,91905,G,19,S,0,H,748,9,2012,0.000000,...,0.0,0.777778,0.222222,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,91906,D,0,D,0,D,0,27,2012,0.000000,...,0.0,0.629630,0.222222,0.074074,0.037037,0.000000,0.000000,0.037037,0.0,0.000000


# PLOT

plotly colors: https://media.geeksforgeeks.org/wp-content/uploads/20220220154706/newplot.png

In [45]:
def generate_zbp_chloropleth(data, year):
    df = data[data['year']==year].groupby('zip')['emp'].sum().reset_index()[['zip','emp']]
    
    with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/ca_california_zip_codes_geo.min.json') as response:
        zipcodes = json.load(response)
    
    fig = px.choropleth(df, 
                        geojson=zipcodes, 
                        locations='zip', 
                        color='emp',
                        color_continuous_scale='blues',
                        range_color=(df['emp'].min(),df['emp'].max()),
                        featureidkey="properties.ZCTA5CE10",
                        scope="usa",
                        labels={'Final_Labels':'Cluster_Category'})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.write_html(f'../../src/data/out/plots/zbp_plot_{year}.html')

    display(IFrame(src=f'../../src/data/out/plots/zbp_plot_{year}.html', width=1000, height=1000))

In [46]:
generate_zbp_chloropleth(data, 2021)